<a href="https://colab.research.google.com/github/EvanWAppel/exploration/blob/master/Time_Discrepancy_Analysis_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Discrepancy Analysis Engine

This Python notebook is designed to compile and analyze several sets of data and to produce tools for discovering discrepancies.

When run, the engine will produce an excel file with its findings.

Production note: Started working on this on September 7. In one month I have a viable product.

## Step 1:

Upload data. 

Files need to be named thusly:

1.   employee_file
2.   exceptions
3.   schedule
4.   project
5.   sessions
6.   slices

This can be accomplished by uploading directly into the content folder or by using the following code to prompt a set of uploads. (Activate the next cell's code to use the prompts.)

## Step 2:

Run the Engine. Click "Runtime" in the toolbar. Then click "Run All."

## Step 3: 

Download the file.

## More information?

For more information, please see the following cell which details specifications. If you can't find what you're looking for, please call Evan Appel at 702-466-4498.





# Documentation

The output Excel file has many sheets in it, each are described here:

1. employeeFile

      File downloaded from Bamboo. Contains basic information about the employee such as their EID, name, date of hire, job title, and division.

2. projectData

      File downloaded from OnTime. Not all data sets have reliable project data and as such, that column cannot be used in every analysis document. Hence why we have an RA Analysis and a Project Analysis.

3. exceptionList
4. schedule
5. voxcoSessions
6. onTimeSlices
7. RAandDateAnalysis
8. projectandRAAnalysis
9. morningPadding
10. eveningPadding
11. excessiveBreaks
12. toomuchinnerpadding
13. attendanceVariance
14. histogramsAndScatterPlots
15. projectComparison
16. departmentAnalysis
18. nonRAAnalysis
19. exception analysis
20. Inconsistent project

#Query References
##Sessions Query

```

 /*TITLE: Agent Sessions Query
   AUTHOR: Evan Appel
   UPDATED: May 7, 2020*/
 
        SELECT LOWER(a.FirstName + ' ' + a.LastName) AS agent
             , LEFT(dirsrv.text,PATINDEX('% %',dirsrv.text)) AS bamboo_id
             , o2.Name AS project_name
             -- The following sets the login and logout times to ISO 8601 for coding purposes
             /*, FORMAT(s.loginDateTime,'yyyy-MM-ddTHH:mm:ss.fffZ') AS LOGIN
             , FORMAT(s.logoutDateTime, 'yyyy-MM-ddTHH:mm:ss.fffZ') AS logout */
             , CAST(DATEADD(ss,DATEDIFF(ss,GETUTCDATE(), GETDATE()),s.loginDateTime) AS DATE) AS LOGIN_DATE
             , CAST(DATEADD(ss,DATEDIFF(ss,GETUTCDATE(), GETDATE()),s.loginDateTime) AS TIME) AS lOGIN_TIME
             , CAST(DATEADD(ss,DATEDIFF(ss,GETUTCDATE(), GETDATE()),s.logoutDateTime) AS DATE) AS LOGOUT_DATE
             , CAST(DATEADD(ss,DATEDIFF(ss,GETUTCDATE(), GETDATE()),s.logoutDateTime) AS TIME) AS LOGOUT_TIME
             , DATEDIFF(ms, s.loginDateTime, s.logoutDateTime)/1000 AS sesh_dur
             , [talkTime]
             , [waitingTime]
             , [pauseTime]
             , [reviewTime]
             , [talkTime] + [waitingTime] + [pauseTime] + [reviewTime] AS soat 
            -- , ap.total AS completes
             --, (((DATEDIFF(ms, s.loginDateTime, s.logoutDateTime)/1000)  / ap.total) / 60) AS prod_rate
           /* */
          FROM [VoxcoSystem].[dbo].[AgentSession] s
     LEFT JOIN [VoxcoSystem].[dbo].[tblAgents] a 
            ON s.userId = a.k_Id
     LEFT JOIN [VoxcoSystem].[dbo].[tblObjects] o 
            ON s.projectId = o.k_Id
     LEFT JOIN [VoxcoSystem].[dbo].[tblObjects] o2 
            ON o.ParentId = o2.k_Id
     LEFT JOIN [VoxcoSystem].[dbo].[History_DirSrvObjName] dirsrv
            ON s.userId = dirsrv.id
     LEFT JOIN [VoxcoSystem].[dbo].[AgentSessionConnectionTimes] c 
            ON s.sessionId = c.sessionId
     --LEFT JOIN (SELECT * FROM [VoxcoSystem].[dbo].[AgentSessionProductivity]
               --  WHERE resultCode = 'CO') ap
           -- ON s.sessionId = ap.sessionId
            -- Date parameter set to yesterday, local time.
         WHERE CAST(DATEADD(ss,DATEDIFF(ss,GETUTCDATE(), GETDATE()),s.loginDateTime) AS DATE) BETWEEN '2021-08-28' AND '2021-09-03' -- weekend code
        -- CAST(DATEADD(ss,DATEDIFF(ss,GETUTCDATE(), GETDATE()),s.loginDateTime) AS DATE) = CAST(getdate() - 1 AS DATE) 
          -- and a.FirstName = 'richard'
      ORDER BY loginDateTime DESC
```
##Slices Query
Use in Cloudwatch, prod event bus

```
fields @timestamp, detail.associate, detail.start, detail.end, detail.minutes, detail.punch, detail.project, `detail.slice-type`
| filter (`detail-type` = "slice") 
and `detail.slice-type` not like "epoch" 
and `detail.punch` not in ["out","manager","courtesy-patrol","monitoring","team-lead-assistant","office-maintenance","office-technician","office-admin","trainer","quota-managemeent",""]
and strlen(`detail.punch`) >=1
| sort @timestamp desc
|limit 10000
```




# Imports

In [1]:
import pandas as pd
import datetime as datetime
import pytz 
from dateutil import parser
import numpy as np
from google.colab import files

# Options and Settings

In [2]:
#Sets options so the data frame doesn't break into "pages"
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 800)
# When I assign a column to itself to change its datatype to datetime it throws a warning, this suppresses that warning. #### USE EXTREME CAUTION
pd.set_option('mode.chained_assignment', None)

# Table Diagnostic Tool

# Ingesters

In [3]:
# Employee Data
try:
  employeeFile = pd.read_csv('employee_file.csv', parse_dates=True)
except:
  print("No Employee File Available")
else:
  print("Employee File checker ...")
  checker = 0
  if len(employeeFile) > 0:
    print("Employee File is non-empty. "+str(len(employeeFile))+" records found.")
  else:
    print("Employee File problem: Empty File! "+str(len(employeeFile))+" records found.")
  for i in employeeFile.columns :
    if i in ['Last name, First name', 'Employee #', 'Status', 'Division', 'Job Title','Hire Date']:
      checker += 1
  if checker == 6:
    print("Required columns present for Employee File")
  else:
    print("Check Employee File document for correct fields, referenced above.")
# Exception Data
try:
  exceptions = pd.read_csv('exceptions.csv', parse_dates=True)
except:
  print("No exceptions file Available")
else:
  print("Exceptions file checker ...")
  if len(exceptions) > 0:
    print("Employee File is non-empty. "+str(len(exceptions))+" reecords found.")
  else:
    print("Employee File is empty! "+str(len(exceptions))+" records found.")
  print("Data begins on "+str(exceptions['date'].min())+"\n"+"And ends on "+str(exceptions['date'].max()))
# Schedule Data
try:
  schedule = pd.read_csv('schedule.csv')
except:
  print("No schedule file Available")
else:
  print("Schedule file checker ...")
  if len(schedule) > 0:
    print("Schedule File is non-empty. "+str(len(schedule))+" reecords found.")
  else:
    print("Schedule File is empty! "+str(len(schedule))+" records found.")
  print("Data begins on "+str(schedule['Date'].min())+"\n"+"And ends on "+str(schedule['Date'].max()))
# Sessions Data
try:
  sessions = pd.read_excel('sessions.xlsx', parse_dates=True)
except:
  print("No sessions file Available")
else:
  print("Sessions file checker ...")
  if len(sessions) > 0:
    print("Sessions File is non-empty. "+str(len(sessions))+" records found.")
  else:
    print("Sessions File is empty! "+str(len(sessions))+" records found.")
  print("Data begins on "+str(sessions['LOGIN_DATE'].min())+"\n"+"And ends on "+str(sessions['LOGIN_DATE'].max()))
# Slices Data
try:
  slices = pd.read_csv('slices.csv', parse_dates=True)
except:
  print("No slices file Available")
else:
  print("Slices file checker ...")
  if len(slices) > 0:
    print("Slices File is non-empty. "+str(len(slices))+" records found.")
  else:
    print("Slices File is empty! "+str(len(schedule))+" records found.")
  print("Data begins on "+str(slices['detail.start'].min())+"\n"+"And ends on "+str(slices['detail.start'].max()))

Employee File checker ...
Employee File is non-empty. 509 records found.
Required columns present for Employee File
Exceptions file checker ...
Employee File is non-empty. 527 reecords found.
Data begins on 2021-09-29
And ends on 2021-10-05
Schedule file checker ...
Schedule File is non-empty. 339 reecords found.
Data begins on 2021-09-29
And ends on 2021-10-05
Sessions file checker ...
Sessions File is non-empty. 550 records found.
Data begins on 2021-09-29 00:00:00
And ends on 2021-10-05 00:00:00
Slices file checker ...
Slices File is non-empty. 9760 records found.
Data begins on 2021-07-28T00:54:22.954075Z
And ends on 2021-10-06T03:37:47.280888Z


In [4]:
def diag(x,y,z):
  # x == dataframe you wish to analyze
  # y == mode you wish to use
  ## y == RA == RA search mode, returns info about an EID entered into the z position
  # z == mode parameter A
  if y == None and z == None:
    print(x.dtypes)
    print(x.describe())
    print(x.head(50))
  if y == "RA":
    try:
      table = x[x["eid"]==z]
      print(table.dtypes)
      print(table.describe())
      print(table.head(50))
    except: 
      print("No eid found")

# Test
#diag(metrics,"RA",100045)

# Table Preparation
This part is to get table data types arranged properly.

In [5]:
### Employee File
# Only Relevant Columns
employee = employeeFile[["Last name, First name","Employee #","Status","Division","Job Title","Hire Date"]]
# Better column names
employee = employee.rename(columns={"Last name, First name":"full_name",
                                    "Employee #": "eid",
                                    "Hire Date": "hire_date"})
# Set hire date to proper date type
employee["hire_date"] = pd.to_datetime(employee["hire_date"])
# Diagnostics


In [6]:
### Exceptions
# Only Relevant columns
excep = exceptions[["eid","date","project","id"]]
# Set Date to proper type
excep["date"] = pd.to_datetime(excep["date"])
# Aggregate
excep = excep.groupby(["eid","date","project"], as_index=False)["id"].size().fillna(0)
excep = excep.rename(columns={"size":"exception_count"})
excep = excep[["eid","date","exception_count"]]
# Diagnostics


In [7]:
### Schedule
# Only Relevant Columns
sched = schedule[["EID","Project Name","Date","StartTime","EndTime"]]
# column names are to be descriptive, undercase and words separated by underscores
sched = sched.rename(columns={"EID":"eid",
                              "Project Name":"sched_project",
                              "Date":"date",
                              "StartTime":"sched_start",
                              "EndTime":"sched_end"})
# Convert the datetimes to proper types. WARNING: Had to disable a warning for the following 3 rows. Note in above section.
sched["sched_start"] = pd.to_datetime(sched["date"] + " " + sched["sched_start"])
sched["sched_end"] = pd.to_datetime(sched["date"] + " " + sched["sched_end"])
sched["date"] = pd.to_datetime(sched["date"])
# Diagnostics


In [8]:
### Sessions
# Relevant Columns
sesh = sessions[["bamboo_id","project_name","LOGIN_DATE","lOGIN_TIME","LOGOUT_DATE","LOGOUT_TIME","sesh_dur","talkTime","waitingTime","pauseTime","reviewTime","soat"]]
# Rename Columns
sesh = sesh.rename(columns={"bamboo_id":"eid",
                            "project_name":"vc_project",
                            "talkTime":"talk_time",
                            "waitingTime":"waiting_time",
                            "pauseTime":"pause_time",
                            "reviewTime":"review_time"})
# Correct Datatypes
# To get the dates working: basically, the date and time need to be converted to strings and then concatenated, from there, a UTC suffix can be added with the first tz_localize, converted to Pacific time, and then remove the suffix with the second tz_localize
sesh["vc_login"] = pd.to_datetime(sesh["LOGIN_DATE"].astype(str) + " " + sesh['lOGIN_TIME'].astype(str)).dt.tz_localize("US/Pacific").dt.tz_localize(None)
sesh["vc_logout"] = pd.to_datetime(sesh["LOGOUT_DATE"].astype(str) + " " + sesh['LOGOUT_TIME'].astype(str)).dt.tz_localize("US/Pacific").dt.tz_localize(None)
sesh["date"] = sesh["vc_login"].dt.date
sesh["eid"] = sesh["eid"].fillna(0)
sesh = sesh[sesh["eid"] != '26045B']
sesh = sesh[sesh["eid"] != 'jleejoyce']
sesh = sesh[sesh["eid"] != 'kbigelow']
sesh["eid"] = sesh["eid"].astype("str").astype("int64")
# Diagnostics



In [9]:
### Slices
# Relevant Columns
sl = slices[["detail.associate","detail.start","detail.end","detail.minutes","detail.punch","detail.project"]]
# Rename Columns
sl = sl.rename(columns={"detail.associate":"eid",
                        "detail.start":"ontime_start",
                        "detail.end":"ontime_end",
                        "detail.minutes":"ontime_duration",
                        "detail.punch":"punch",
                        "detail.project":"ontime_project"})
# Data Type Correction
sl["eid"] = sl["eid"].astype("int64")
sl["ontime_start"] = pd.to_datetime(sl["ontime_start"]).dt.tz_convert("US/Pacific").dt.tz_localize(None)
sl["ontime_end"] = pd.to_datetime(sl["ontime_end"]).dt.tz_convert("US/Pacific").dt.tz_localize(None)
sl["date"] = sl["ontime_start"].dt.date
# Data Cleaning
# Rows with NAN in certain columns indicate a correction that hasn't been filtered by Cloudwatch
sl = sl.dropna()
#Diagnostics


# Metrics

In [10]:
################################################################### Schedule
# The entire table is based on who should be here, which is determined by the schedule
metrics = sched[["eid","sched_project","date","sched_start","sched_end"]]

################################################################### Employee
# Add employee information
metrics = pd.merge(metrics,employee,on=["eid"],how="left")

################################################################### Exceptions
# Add in how many exceptions they filed
metrics = pd.merge(metrics,excep,on=["eid","date"],how="left").fillna(0)

################################################################### Voxco
# First Voxco Login
# Figure out the first voxco login by person/date
firstVC = sesh.groupby(["eid","date"],as_index=False).agg({"vc_login":min})
# prepare the list of projects
vcProjectIn = sesh[["eid","date","vc_login","vc_project" ]]
# rename project
vcProjectIn = vcProjectIn.rename(columns={"vc_project":"vc_login_project"})
# Join the project to the first login of the day so you only have the project for the first login
firstVC = pd.merge(firstVC,vcProjectIn,on=["eid","date","vc_login"],how="left")
# convert firstVC date to proper format
firstVC["date"] = pd.to_datetime(firstVC["date"])
# Join first login info
metrics = pd.merge(metrics,firstVC,on=["eid","date"],how="left")

# Last Voxco Logout
# Last logout 
lastVC = sesh.groupby(["eid","date"],as_index=False).agg({"vc_logout":max})
# Last logout project
vcProjectOut = sesh[["eid","date","vc_logout","vc_project" ]]
# rename project
vcProjectOut = vcProjectOut.rename(columns={"vc_project":"vc_logout_project"})
# Join project to logout
lastVC = pd.merge(lastVC,vcProjectOut,on=["eid","date","vc_logout"],how="left")
# convert lastVC date to proper format
lastVC["date"] = pd.to_datetime(lastVC["date"])
# Join last logout info
metrics = pd.merge(metrics,lastVC,on=["eid","date"],how="left")

# Voxco Durations
# sum up durations, project agnostic to fit the table
vcTimes = sesh.groupby(["eid","date"],as_index=False).agg({"sesh_dur":sum,
                                                           "soat":sum,
                                                           "talk_time":sum,
                                                           "waiting_time":sum,
                                                           "pause_time":sum,
                                                           "review_time":sum})
# Convert vcTimes date to correct format
vcTimes["date"] = pd.to_datetime(vcTimes["date"])
# Join durations
metrics = pd.merge(metrics,vcTimes,on=["eid","date"],how="left")

# Voxco Count
# count the sessions
vcCount = sesh.groupby(["eid","date"],as_index=False)["eid"].size()
# Convert vcCount date to proper format
vcCount["date"] = pd.to_datetime(vcCount["date"])
vcCount = vcCount.rename(columns={"size":"vc_session_count"})
# Join count
metrics = pd.merge(metrics,vcCount,on=["eid","date"],how="left")

################################################################### Slices
# First OnTime Punch
# Figure out the first OnTime Punch by person/date
firstOT = sl.groupby(["eid","date"],as_index=False).agg({"ontime_start":min})
# prepare the list of projects
otProjectIn = sl[["eid","date","ontime_start","ontime_project" ]]
# rename project
otProjectIn = otProjectIn.rename(columns={"ontime_project":"ontime_start_project"})
# Join the project to the first punch of the day so you only have the project for the first punch
firstOT = pd.merge(firstOT,otProjectIn,on=["eid","date","ontime_start"],how="left")
# convert firstVC date to proper format
firstOT["date"] = pd.to_datetime(firstOT["date"])
# Join first login info
metrics = pd.merge(metrics,firstOT,on=["eid","date"],how="left")

# Last OnTime Punch
# Figure out the last OnTime Punch by person/date
lastOT = sl.groupby(["eid","date"],as_index=False).agg({"ontime_end":max})
# prepare the list of projects
otProjectOut = sl[["eid","date","ontime_end","ontime_project" ]]
# rename project
otProjectOut = otProjectOut.rename(columns={"ontime_project":"ontime_end_project"})
# Join the project to the first punch of the day so you only have the project for the first punch
lastOT = pd.merge(lastOT,otProjectOut,on=["eid","date","ontime_end"],how="left")
# convert firstVC date to proper format
lastOT["date"] = pd.to_datetime(lastOT["date"])
# Join first login info
metrics = pd.merge(metrics,lastOT,on=["eid","date"],how="left")

# Ontime Durations
# sum up durations, project agnostic to fit the table
otTimes = sl.groupby(["eid","date"],as_index=False).agg({"ontime_duration":sum})
# Convert otTimes date to correct format
otTimes["date"] = pd.to_datetime(otTimes["date"])
# Join durations
metrics = pd.merge(metrics,otTimes,on=["eid","date"],how="left")

# OT Count
# count the sessions
otCount = sl.groupby(["eid","date"],as_index=False)["eid"].size()
# Convert otCount date to proper format
otCount["date"] = pd.to_datetime(otCount["date"])
otCount = otCount.rename(columns={"size":"ontime_slice_count"})
# Join count
metrics = pd.merge(metrics,otCount,on=["eid","date"],how="left")

# OnTime Breaks
breakData = sl[sl["punch"]=="break"]
breakCount = breakData.groupby(["eid","date"],as_index=False).size()
breakSum = breakData.groupby(["eid","date"],as_index=False).agg({"ontime_duration":sum})
breakData = pd.merge(breakCount,breakSum,on=["eid","date"],how="left")
breakData = breakData.rename(columns={"ontime_duration":"break_duration",
                                      "size":"break_count"})
breakData["date"] = pd.to_datetime(breakData["date"])
metrics = pd.merge(metrics,breakData,on=["eid","date"],how="left")

################################################################### MOAR COLUMNS!!!
# Diff between schedule start and ontime start
metrics = metrics.assign(schedule_to_ontime=metrics["ontime_start"] - metrics["sched_start"])
metrics["schedule_to_ontime"] = metrics["schedule_to_ontime"].dt.total_seconds()
metrics["schedule_to_ontime"] = metrics["schedule_to_ontime"] / 60
# Diff between ontime start and voxco start
metrics = metrics.assign(ontime_to_voxco=metrics["vc_login"] - metrics["ontime_start"])
metrics["ontime_to_voxco"] = metrics["ontime_to_voxco"].dt.total_seconds()
metrics["ontime_to_voxco"] = metrics["ontime_to_voxco"]/60
# Diff between voxco end and ontime end
metrics = metrics.assign(voxco_to_ontime=metrics["ontime_end"] - metrics["vc_logout"])
metrics["voxco_to_ontime"] = metrics["voxco_to_ontime"].dt.total_seconds()
metrics["voxco_to_ontime"] = metrics["voxco_to_ontime"]/60
# Diff between ontime end and schedule end
metrics = metrics.assign(ontime_to_schedule=metrics["sched_end"] - metrics["ontime_end"])
metrics["ontime_to_schedule"] = metrics["ontime_to_schedule"].dt.total_seconds()
metrics["ontime_to_schedule"] = metrics["ontime_to_schedule"] /60

# "sesh_dur" to mins
metrics["sesh_dur"] = metrics["sesh_dur"]/60
# "soat" to mins
metrics["soat"] = metrics["soat"]/60
# "talk_time" to mins
metrics["talk_time"] = metrics["talk_time"]/60
# "waiting_time" to mins
metrics["waiting_time"] = metrics["waiting_time"]/60
# "pause_time" to mins
metrics["pause_time"] = metrics["pause_time"]/60
# "review_time" to mins
metrics["review_time"] = metrics["review_time"]/60
# "schedule_duration" to mins
metrics = metrics.assign(schedule_duration=metrics["sched_end"] - metrics["sched_start"])
metrics["schedule_duration"] = metrics["schedule_duration"].dt.total_seconds()
metrics["schedule_duration"] = metrics["schedule_duration"] / 60
# Work Rate
metrics = metrics.assign(work_rate=metrics["ontime_duration"] / metrics["schedule_duration"])
# Absence Rate
metrics = metrics.assign(absence_rate=metrics["ontime_duration"] / (metrics["schedule_duration"])-1)
# Potential Discrepancy
metrics = metrics.assign(potential_discrepancy=metrics["schedule_duration"] - metrics["sesh_dur"])


################################################################### Reorder table
metrics = metrics[["full_name",
                   "eid",
                   "Job Title",
                   "Division",
                   "Status",
                   "hire_date",
                   "date",
                   "potential_discrepancy",
                   "work_rate",
                   "absence_rate",
                   "exception_count",
                   "sched_start",
                   "sched_project",
                   "schedule_to_ontime",
                   "ontime_start",
                   "ontime_start_project",
                   "ontime_to_voxco",
                   "vc_login",
                   "vc_login_project",
                   "vc_logout",
                   "vc_logout_project",
                   "voxco_to_ontime",
                   "ontime_end",
                   "ontime_end_project",
                   "ontime_to_schedule",
                   "sched_end",
                   "sched_project",
                   "talk_time",
                   "waiting_time",
                   "pause_time",
                   "review_time",
                   "sesh_dur",
                   "soat",
                   "ontime_duration",
                   "schedule_duration",
                   "vc_session_count",
                   "ontime_slice_count",
                   "break_duration",
                   "break_count"]]
################################################################### Pre-filters

################################################################### Diagnostics
print(metrics)

              full_name     eid           Job Title Division  Status            hire_date       date  potential_discrepancy  work_rate  absence_rate  exception_count         sched_start sched_project  schedule_to_ontime               ontime_start ontime_start_project  ontime_to_voxco                vc_login vc_login_project               vc_logout vc_logout_project  voxco_to_ontime                 ontime_end ontime_end_project  ontime_to_schedule           sched_end sched_project   talk_time  waiting_time  pause_time  review_time    sesh_dur        soat  ontime_duration  schedule_duration  vc_session_count  ontime_slice_count  break_duration  break_count
0            Ross, Alex  100045  Research Associate      CRD  Active  2021-07-13 00:00:00 2021-09-29             231.866667   0.432000     -0.568000              0.0 2021-09-29 13:45:00       kf1001c           15.274074 2021-09-29 14:00:16.444414              kf1001c         3.053043 2021-09-29 14:03:19.627          KF1001C 2021-09-29 

# Concern List

In [11]:
# This code block produces a list of concerning circumstances for Workforce Management to address
# The first part establishes some parameters for the concerns list
# The second part creates the list

#Parameters
#How many exceptions trigger an event?
## The first standard deviation of the mean rounded plus one
exception_trigger = abs(round(metrics["exception_count"].std())) + 1
# What defines a concerning discrepancy?
## Greater than the absolute value of one standard deviation from the mean?
discrepancy_trigger = abs(round(metrics["potential_discrepancy"].std()))
# Work Rate
work_rate_trigger = 0.7
# Schedule to Ontime
sched_to_ontime_trigger = 5



# Flagger

## Exceptions
ex = metrics[metrics["exception_count"] >= exception_trigger]
ex["exception_count"] = ex["exception_count"].astype("int64")
ex = ex.assign(message="Employee has "+ex["exception_count"].astype("str")+" exceptions. "+(ex["exception_count"]-(exception_trigger-1)).astype("str")+" more than allowance.")
ex = ex[["full_name","eid","date","message"]]

# Discrepancy
di = metrics[abs(metrics["potential_discrepancy"]) > discrepancy_trigger]
di = di.assign(message="Employee has "+round(di["potential_discrepancy"]).astype("int64").astype("str")+" discrepancy minutes. "+"The parameter is "+str(discrepancy_trigger)+" minutes.")
di = di[["full_name","eid","date","message"]]

# Work Rate



# Schedule to Ontime
so = metrics[abs(metrics["schedule_to_ontime"])>sched_to_ontime_trigger]
so = so.assign(message="Employee has exceeded the " + str(sched_to_ontime_trigger) + " minute schedule to ontime trigger by " + str(metrics["schedule_to_ontime"] - sched_to_ontime_trigger) + " minutes." )
so = so[["full_name","eid","date","message"]]
print(so.head())


# Ontime to Voxco



# Voxco to Ontime



# Ontime to Schedule



# Too many breaks



# Too many break minutes






# Concerns
concerns = pd.concat([ex,di,so])
print(concerns.head())


         full_name     eid       date                                            message
0       Ross, Alex  100045 2021-09-29  Employee has exceeded the 5 minute schedule to...
1      Ayala, Eric  100412 2021-09-29  Employee has exceeded the 5 minute schedule to...
2    Luna, Rolando  100548 2021-09-29  Employee has exceeded the 5 minute schedule to...
9   Rascon, Orlena  102713 2021-09-29  Employee has exceeded the 5 minute schedule to...
11  Tyrell, Shyann  102904 2021-09-29  Employee has exceeded the 5 minute schedule to...
               full_name     eid       date                                            message
10  Betancourt, Jennifer  102872 2021-09-29  Employee has 2 exceptions. 1 more than allowance.
85          Mayne, Kelly  105124 2021-09-30  Employee has 3 exceptions. 2 more than allowance.
86          Mayne, Kelly  105124 2021-09-30  Employee has 3 exceptions. 2 more than allowance.
87    Ifopo, Christopher  105171 2021-09-30  Employee has 2 exceptions. 1 more than al

# Concerns Summary

In [12]:
concernsSummary = concerns.groupby(["full_name","eid"]).size()
print(concernsSummary.head())

full_name         eid   
0                 105380    1
Ayala, Eric       100412    8
Bailey, Geneva    105318    2
Belt, Tamara      23928     5
Benitez, Minerva  105478    3
dtype: int64


## Sample code for creating an excel file with multiple tabs
```
with pd.ExcelWriter('two_frames_one_tab.xlsx', engine='xlsxwriter') as writer:
    df_first.to_excel(writer, sheet_name='first_tab', index=False)
    df_second.to_excel(writer, sheet_name='second_tab', index=False)
```
## How to download an excel file
files.download('example.xlsx') invokes a browsere download of the file to the local machine.
```
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')
```




In [13]:
# Now, make an excel doc with all the data in it and download
with pd.ExcelWriter('Analysis_Engine.xlsx',
                    datetime_format = 'YYYY-MM-DD HH:MM:SS') as writer:
                    # Must make dates Timezone unaware!
                    # RAData["Start"] = RAData["Start"].dt.tz_localize(None
                    # RAData["End"] = RAData["End"].dt.tz_localize(None)
  employee.to_excel(writer, sheet_name='employeeFile', index=False)
  excep.to_excel(writer, sheet_name='exceptionList', index=False)
  sched.to_excel(writer, sheet_name='schedule', index=False)
  sesh.to_excel(writer, sheet_name='voxcoSessions', index=False)
  sl.to_excel(writer, sheet_name='onTimeSlices', index=False)
  metrics.to_excel(writer,sheet_name='metrics',index=False)
  concerns.to_excel(writer,sheet_name='concerns',index=False)
  concernsSummary.to_excel(writer,sheet_name='concernSummary',index=False)
#Download Step
files.download('Analysis_Engine.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>